In [2]:
import numpy as np

In [7]:
def nelder_mead(f, x_start, step=0.5, tol=1e-6, max_iter=500):
    n = len(x_start)
    # Initialize simplex
    simplex = [x_start]
    for i in range(n):
        x = np.array(x_start, dtype=float)
        x[i] += step
        simplex.append(x)
    simplex = np.array(simplex)
    print(simplex)
    for iteration in range(max_iter):
        # Sort simplex points by function value
        simplex = sorted(simplex, key=f)
        f_values = [f(x) for x in simplex]

        # Check convergence
        if np.std(f_values) < tol:
            break

        # Compute centroid (excluding worst point)
        centroid = np.mean(simplex[:-1], axis=0)

        # Reflection
        xr = centroid + (centroid - simplex[-1])
        fr = f(xr)

        if f_values[0] <= fr < f_values[-2]:
            simplex[-1] = xr
        elif fr < f_values[0]:
            # Expansion
            xe = centroid + 2 * (xr - centroid)
            fe = f(xe)
            simplex[-1] = xe if fe < fr else xr
        else:
            # Contraction
            xc1 = simplex[-1] + 0.25 * (xr-simplex[-1] )   
            xc3 = simplex[-1] + 0.75 * (xr-simplex[-1] )
            fc1 = f(xc1)
            fc3 = f(xc3)
            xc = 0
            fc = 0
            if fc1 < fc3:
                xc,fc= xc1,fc1
            else:
                xc,fc = xc3,fc3
            if fc < f_values[-1]:
                simplex[-1] = xc1
            else:
                # Shrink
                for i in range(1, len(simplex)):
                    simplex[i] = simplex[0] + 0.5 * (simplex[i] - simplex[0])

    best = min(simplex, key=f)
    return best, f(best), iteration + 1


In [7]:
def funcheck(v):
    x,y = v
    return (x - 1)**2 + (y - 2)**2  


In [11]:
#creating a noisy function
K = 10
alpha = 0
np.random.seed(27)
phi = 1 * np.pi * np.random.rand(K, K)  # fixed phases

# Function f(v) = sin(x) + y^2 + noise
#changed to f(v) = x^2 + y^2 + noise
def f_structured(v):
    x, y = v
    #f_val= np.sin(x) + y**2
    f_val = x**2 + y**2
    for k1 in range(K):
        for k2 in range(K):
            amp = 1.0 / (1 + k1**2 + k2**2)**(alpha / 2)
            f_val += 0.05*(amp * np.cos(k1 * x + k2 * y + phi[k1, k2]))
    return f_val


In [12]:
x0 = np.array([10,-10])
sol, fval, iters = nelder_mead(f_structured, x0)
print(f"Minimum at {sol}, value = {fval}, iterations = {iters}")

[[ 10.  -10. ]
 [ 10.5 -10. ]
 [ 10.   -9.5]]
Minimum at [-0.72239063 -0.50481414], value = 0.21471236957956785, iterations = 42
